In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from typing import Dict, Union, Callable
from sklearn.linear_model import LinearRegression
from dataclasses import dataclass
import scipy.stats
import graphs
from enum import Enum

## Having a look at the data

In [2]:
dataset = pd.read_csv('trainset.csv')
#_,ax = plt.subplots(nrows=len(dataset.columns), ncols=1, figsize=(10,12))
#plt.tight_layout()
#for i,column in enumerate(dataset.columns):
#    ax[i].hist(dataset[column])
#    ax[i].set_xlabel(column)

# I'll start by considering quality as a continuous variable with no further changes
data = dataset.to_numpy()

## Learning a linear gaussian bayes net with a given adjacency matrix

A linear gaussian bayes net describes a gaussian probability distribution. Each node in such a network itself represents a gaussian distribution over one attribute/variable. Edges between nodes signify dependence of one variable on the other. ($A \rightarrow B$ means $A$ influences $B$, $B$ is dependent on $A$)

For each node $x_i$ , we define its conditional probability distribution:
$$
p(x_i | \text{parents}(i)) = \mathcal{N}\left( \beta_{i0} + \sum_{i \in \text{parents}(j)} \beta_{ij} x_j, \sigma_i \right)
$$
with parameters:
- $\beta_{ij}$: coefficients describing the way $x_j$ influences $x_i$
- $\beta_{i0}$: offset 
- $\sigma_i$: conditional variance of $x_i$ (conditioned on $\text{parents}(i)$)

We compute the coefficients and offset by fitting a linear regression, and compute the conditional variance using a formula derived from its definition:
$$
\text{Var}[Y|X] = E \left[(Y - E[Y|X])^2 \right] \\
\text{sampleVar}(y|x) = \frac{1}{m-1} \sum_{i=1}^m (y_i - \hat{y}_x)^2
$$
where:
- $y_i$ is the value of this attribute for each datapoint
- $\hat{y}_x$ is the predicted value of $y_i$ given the values of the other attributes $x$

##### Likelihood:
Given some data $X = (x^{(1)}, x^{(2)}, ..., x^{(m)})$:
$$
L(S) = \prod_{i=1}^m p(x^{(i)} | S, \theta)
$$
where $S$ represents the model structure (in our case, the adjacency matrix), and $\theta$ represents the learned parameters.
We only view the likelihood as a function of $S$.
We can write down the likelihood for our models in short:
$$
L(\text{parents}) = \prod_{j=1}^m \prod_{i=1}^n p(x_i^{(j)} | \text{parents}(x_i)^{(j)}) = \prod_{j=1}^m \prod_{i=1}^n  \mathcal{N}\left( \beta_{i0} + \sum_{i \in \text{parents}(k)} \beta_{ik} x_k^{(j)}, \sigma_i \right)
$$

In [3]:
@dataclass
class StaticParam:
    mu: float
    sigma: float

@dataclass
class DependentParam:
    coefficients: np.ndarray
    offset: float
    sigma: float

Param = Union[StaticParam, DependentParam] # A param is either static or dependent

class GaussNet():
    def __init__(self, adj: np.ndarray):
        self.adj = adj
        self.n = adj.shape[0]  # number of variables
        self.params: Dict[int, Param] = {}  # parameters defining gaussian distribution for each variable
    
    # mode=0 uses explicit linear regressions from sklearn while
    # mode=1 uses computations using the covariance matrix with no further packages used
    def fit(self, data: np.ndarray, mode=1):
        if mode == 1:
            return self.__fit1(data)
        else:
            return self.__fit2(data)
    
    def __fit1(self, data: np.ndarray):
        def compute_static_params():
            target_data = data[:, node_idx]
            mu = np.mean(target_data)
            sigma = np.var(target_data)
            return StaticParam(mu, sigma)
            
        def compute_dependent_params():
            target_data = data[:, node_idx]
            predictor_data = data[:, graphs.parents(self.adj, node_idx)]
            reg = LinearRegression().fit(predictor_data, target_data) # fit a linear regression
            coefficients = reg.coef_
            offset = reg.intercept_
            predicted_data = reg.predict(predictor_data)
            sigma = (1/(m-1)) * np.sum((target_data - predicted_data)**2)
            return DependentParam(coefficients, offset, sigma)
        
        m = data.shape[0]
        for node_idx in range(self.n):
            if graphs.parents(self.adj, node_idx) == []:  # No parents
                self.params[node_idx] = compute_static_params()
            else:
                self.params[node_idx] = compute_dependent_params()
        return self
    
    def __fit2(self, data: np.ndarray):
        def compute_static_params() -> StaticParam:
            target_data = data[:, node_idx]
            mu = np.mean(target_data)
            sigma = np.var(target_data)
            return StaticParam(mu, sigma)
        def compute_dependent_params() -> DependentParam:
            target_data = data[:, node_idx]
            parents = graphs.parents(self.adj, node_idx)
            mu_I = np.mean(target_data)
            sigma_full = np.cov(data, rowvar=False)
            sigma_IJ = self.__myindex(sigma_full, node_idx, parents)
            sigma_JJ = self.__myindex(sigma_full, parents, parents)
            mu_J = np.mean(data[:,parents], axis=0)
            sigma_II = self.__myindex(sigma_full, node_idx, node_idx)
            sigma_JI = self.__myindex(sigma_full, parents, node_idx)
            
            offset = np.squeeze(mu_I - sigma_IJ @ np.linalg.inv(sigma_JJ) @ np.atleast_2d(mu_J).T)
            coefficients = sigma_IJ @ np.linalg.inv(sigma_JJ)
            sigma = np.squeeze(sigma_II - coefficients @ sigma_JI)
            return DependentParam(coefficients.reshape(len(parents)), offset.item(), sigma.item())
        for node_idx in range(self.n):
            if graphs.parents(self.adj, node_idx) == []:  # No parents
                self.params[node_idx] = compute_static_params()
            else:
                self.params[node_idx] = compute_dependent_params()
        return self

    def predict(self, data: np.ndarray, node_idx: int, toround=False):
        predictors = data[:, graphs.parents(self.adj, node_idx)]
        param = self.params[node_idx]
        if type(param) == StaticParam:
            if toround:
                return np.repeat(np.around(param.mu), data.shape[0])
            else:
                return np.repeat(param.mu, data.shape[0])
        else:
            if toround:
                return np.around(param.offset + predictors @ param.coefficients)
            else:
                return param.offset + predictors @ param.coefficients

    def accuracy(self, data: np.ndarray, node_idx: int): # Assumes that node_idx is a categorical variable obtained by rounding the estimate
        predicted = np.reshape(self.predict(data, node_idx, toround=True), data.shape[0])
        return np.count_nonzero(predicted == data[:,node_idx]) / data.shape[0]

    def loglikelihood(self, data: np.ndarray):
        acc = 0
        for node_idx in range(self.n):
            mus = np.squeeze(self.predict(data, node_idx))
            acc += np.sum(scipy.stats.norm.logpdf(data[:,node_idx], mus, self.params[node_idx].sigma))
        return acc
    
    def num_params(self):
        acc = 0
        for _,param in self.params.items():
            if type(param) == StaticParam:
                acc += 2
            else:
                acc += 2 + param.coefficients.shape[0]
        return acc
    
    def __myindex(self, arr, idx_I, idx_J=[]): # Leaves only the indices idx_I and idx_J in array arr
        if arr.ndim > 2: return arr
        if arr.ndim == 1: return arr[idx_I]
        if arr.ndim == 2: return np.atleast_2d(arr[idx_I,:])[:,idx_J]

#### We'll try gauss nets with both modes

In [7]:
num_attributes = data.shape[1]
test_adj = np.zeros((num_attributes, num_attributes))
test_adj[10,11] = 1 # let's say that residual sugar predicts quality
net1 = GaussNet(test_adj)
net1.fit(data)
net1.params[11]

DependentParam(coefficients=array([0.35419866]), offset=1.9564383703769535, sigma=0.5232316131752069)

In [8]:
num_attributes = data.shape[1]
test_adj = np.zeros((num_attributes, num_attributes))
test_adj[10,11] = 1 # let's say that residual sugar predicts quality
net2 = GaussNet(test_adj)
net2.fit(data, mode=2) # Using the other mode this time
net2.params[11]

DependentParam(coefficients=array([0.35419866]), offset=1.9564383703769535, sigma=0.5232316131752068)

In [9]:
print(net1.accuracy(data, 11))
print(net2.accuracy(data, 11))
print(net1.loglikelihood(data))
print(net2.loglikelihood(data))

0.5504587155963303
0.5504587155963303
-169945633.35283077
-169945633.35283077


#### Both modes seem are equivalent!

### For accuracy, only the last column matters, so let's just try all possibilities

That's gonna be $2^{11}=2048$ possibilites.
We'll save the maximum accuracy achieved for each number of attributes used in prediction.

In [10]:
columns = np.array(list(itertools.product([0,1], repeat=11)))
columns = np.hstack((columns, np.atleast_2d(np.zeros(2**11)).T)) # add a 0, so we dont predict quality by quality

max_acc = {}
max_acc_idx = {}
for i in range(12):
    max_acc[i] = -1
    max_acc_idx[i] = -1

for idx,column in enumerate(columns):
    adj = np.zeros((data.shape[1], data.shape[1]))
    adj[:,11] = column
    acc = GaussNet(adj).fit(data).accuracy(data, 11)
    num_attr = np.count_nonzero(column)
    if acc > max_acc[num_attr]:
        max_acc[num_attr] = acc
        max_acc_idx[num_attr] = idx

for i in range(12):
    column = columns[max_acc_idx[i]]
    print(f'Maximum accuracy for {i} attributes used: {max_acc[i]}, achieved using:')
    print(column)
    adj = np.zeros((data.shape[1], data.shape[1]))
    adj[:,11] = column

Maximum accuracy for 0 attributes used: 0.390325271059216, achieved using:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Maximum accuracy for 1 attributes used: 0.5504587155963303, achieved using:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Maximum accuracy for 2 attributes used: 0.5646371976647206, achieved using:
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
Maximum accuracy for 3 attributes used: 0.5746455379482902, achieved using:
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
Maximum accuracy for 4 attributes used: 0.5738115095913261, achieved using:
[0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
Maximum accuracy for 5 attributes used: 0.5863219349457881, achieved using:
[0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.]
Maximum accuracy for 6 attributes used: 0.5863219349457881, achieved using:
[0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0.]
Maximum accuracy for 7 attributes used: 0.5888240200166805, achieved using:
[0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0.]
Maximum accuracy for 8 attributes used: 0.5879899916597164, achieved using:
[1. 1. 1. 0. 

# Structure Learning
## Score-Based Algorithms
For the next algorithms, we will be adding edges to, and removing them from, the graph.
When adding edges, we need to make sure we're keeping the DAG structure.
An added edge can cause a cycle to appear.
In this case, the new edge $A \rightarrow B$ will be part of the cycle.
A cycle will appear if and only if there was a path from $B$ to $A$ in the DAG before the edge was added.
We can check for such a path with a search (in this case, DFS) starting from $B$.

In [11]:
# Some test cases for finding paths in a graph
test_adj = np.zeros((5,5))
print(f'1: {graphs.path_exists(test_adj, 0, 1)}')
print(f'2: {graphs.path_exists(test_adj, 0, 0)}')
test_adj[0,3] = 1
print(f'3: {graphs.path_exists(test_adj, 0, 3)}')
print(f'4: {graphs.path_exists(test_adj, 3, 0)}')
test_adj[3,4] = 1
print(f'4: {graphs.path_exists(test_adj, 0, 4)}')
print(f'5: {graphs.path_exists(test_adj, 4, 3)}')
print(f'6: {graphs.path_exists(test_adj, 4, 0)}')
test_adj[3,2] = 1
test_adj[3,1] = 1
test_adj[4,4] = 1
test_adj[4,1] = 1
test_adj[4,2] = 1
print(f'7: {graphs.path_exists(test_adj, 0, 4)}')
print(f'8: {graphs.path_exists(test_adj, 4, 3)}')
print(f'9: {graphs.path_exists(test_adj, 4, 0)}')

1: False
2: True
3: True
4: False
4: True
5: False
6: False
7: True
8: False
9: False


## Scores
### Bayesian Information Criterion

In [12]:
def BIC(loglikelihood: float, num_params: int, num_datapoints: int) -> float:
    return num_params*np.log(num_datapoints) - 2*loglikelihood

### Akaike Information Criterion

In [13]:
def AIC(loglikelihood: float, num_params: int, num_datapoints: int) -> float:
    return 2*num_params - 2*loglikelihood

## Search Algorithms
### Local Additive Search
We start with an empty graph (no edges) and gradually add the legal edge that brings the highest score-increase, until there is none.

In [26]:
def local_additive_search(data: np.ndarray, metric: Callable) -> np.ndarray:
    """
    Performs a local additive search to find an adjacency matrix for a
    linear gaussian network that fits the data well.
    It uses 'metric' as a score to evaluate gaussian networks.
    'metric' should take 3 parameters: 
        the loglikelihood, the number of parameters and the number of datapoints
    """
    n = data.shape[1]
    adj = np.zeros((n,n))  # start with empty adjacency matrix
    net = GaussNet(adj).fit(data)
    highest_score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
    found_better_edge = True
    
    while found_better_edge:
        found_better_edge = False
        legal_edges = [(i,j) for i in range(n) for j in range(n) if adj[i,j]==0 and not graphs.path_exists(adj, j, i)]
        for edge in legal_edges:
            adj[edge] = 1  # add edge to graph
            net = GaussNet(adj).fit(data)
            score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
            if score > highest_score:  # found a better edge
                highest_score = score 
                found_better_edge = True
                better_edge = edge
            adj[edge] = 0
        if found_better_edge:
            adj[better_edge] = 1
    return adj

In [27]:
adj_additive_bic = local_additive_search(data, BIC)
np.save(f'submissions/additive_bic.npy', adj_additive_bic)
net = GaussNet(adj_additive_bic).fit(data)
print(f'Likelihood: {net.loglikelihood(data)} with {net.num_params()} parameters.')

Likelihood: -1069713998.859003 with 79 parameters.


In [28]:
adj_additive_aic = local_additive_search(data, AIC)
np.save(f'submissions/additive_aic.npy', adj_additive_aic)
net = GaussNet(adj_additive_aic).fit(data)
print(f'Likelihood: {net.loglikelihood(data)} with {net.num_params()} parameters.')

Likelihood: -1069714003.838056 with 77 parameters.


### Local Combined Search
We start with an empty graph and in each step either add or remove an edge.
We choose the action that brings the highest score-increase, until there is none.

In [36]:
def local_combined_search(data: np.ndarray, metric: Callable) -> np.ndarray:
    """
    Performs a local combined search to find an adjacency matrix for a
    linear gaussian network that fits the data well.
    It uses 'metric' as a score to evaluate gaussian networks.
    'metric' should take 3 parameters: 
        the loglikelihood, the number of parameters and the number of datapoints
    """
    
    class EdgeAction(Enum):
        ADD = 1
        REMOVE = 2
        
    def do_action(adj, edge_action):
        edge = edge_action[0]
        action = edge_action[1]
        if action == EdgeAction.ADD:
            adj[edge] = 1  # add edge to graph
        else:
            adj[edge] = 0  # remove edge from graph
        return adj
            
    def undo_action(adj, edge_action):
        edge = edge_action[0]
        action = edge_action[1]
        if action == EdgeAction.ADD:
            adj[edge] = 0  # add edge to graph
        else:
            adj[edge] = 1  # remove edge from graph
        return adj
    
    n = data.shape[1]
    adj = np.zeros((n,n))  # start with empty adjacency matrix
    net = GaussNet(adj).fit(data)
    highest_score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
    found_better_action = True
    
    while found_better_action:
        found_better_action = False
        edge_adds = [((i,j), EdgeAction.ADD) for i in range(n) for j in range(n) if adj[i,j]==0 and not graphs.path_exists(adj, j, i)]
        edge_removes = [((i,j), EdgeAction.REMOVE) for i in range(n) for j in range(n) if adj[i,j]==1]
        legal_edge_actions = edge_adds + edge_removes
        for (edge, edge_action) in legal_edge_actions:
            adj = do_action(adj, (edge, edge_action))
            net = GaussNet(adj).fit(data)
            score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
            if score > highest_score:  # found a better action
                highest_score = score 
                found_better_action = True
                better_action = (edge, edge_action)
            adj = undo_action(adj, (edge, edge_action))
        if found_better_action:
            adj = do_action(adj, better_action)
    return adj

In [37]:
adj_combined_bic = local_combined_search(data, BIC)
np.save(f'submissions/combined_bic.npy', adj_combined_bic)
net = GaussNet(adj_combined_bic).fit(data)
print(f'Likelihood: {net.loglikelihood(data)} with {net.num_params()} parameters.')

Likelihood: -1069713998.859003 with 79 parameters.


In [38]:
adj_combined_aic = local_combined_search(data, AIC)
np.save(f'submissions/combined_aic.npy', adj_combined_aic)
net = GaussNet(adj_combined_aic).fit(data)
print(f'Likelihood: {net.loglikelihood(data)} with {net.num_params()} parameters.')

Likelihood: -1069714003.838056 with 77 parameters.


### Simulated Annealing

In [53]:
def simulated_annealing(data: np.ndarray, metric: Callable, temps: np.ndarray, seed: int):
    """
    Performs simulated annealing to find an adjacency matrix for a
    linear gaussian network that fits the data well.
    It uses 'metric' as a score to evaluate gaussian networks.
    'metric' should take 3 parameters: 
        the loglikelihood, the number of parameters and the number of datapoints
    'temps' should be a declining series of positive numbers, each indicating
    the freedom of the algorithm to choose a locally worse solution in search of a better optimum.
    Once the values in 'temps' are exhausted, the algorithm terminates.
    """
    
    class EdgeAction(Enum):
        ADD = 1
        REMOVE = 2
        
    def do_action(adj, edge_action):
        edge = edge_action[0]
        action = edge_action[1]
        if action == EdgeAction.ADD:
            adj[edge] = 1  # add edge to graph
        else:
            adj[edge] = 0  # remove edge from graph
        return adj
    
    def random_action(rng, adj, n) -> ((int,int), EdgeAction):
        found = False
        while not found:
            i, j = rng.integers(0, n, 2)
            if adj[i,j] == 1:
                action = EdgeAction.REMOVE
                found = True
            else:
                action = EdgeAction.ADD
                if not graphs.path_exists(adj, j, i):
                    found = True
        return ((i,j), action)
            
    rng = np.random.default_rng(seed)
    n = data.shape[1]
    adj = np.zeros((n,n))  # start with empty adjacency matrix
    net = GaussNet(adj).fit(data)
    current_score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
    highest_score = current_score
    best_adj = adj.copy()
    
    for temp in temps:
        edge, edge_action = random_action(rng, adj, n)
        net = GaussNet(adj).fit(data)
        score = metric(net.loglikelihood(data), net.num_params(), data.shape[0])
        if score > current_score:
            current_score = score
            adj = do_action(adj, (edge, edge_action))
        else:
            print(- (score - highest_score) / temp)
            if rng.random() < np.exp(- (score - highest_score) / temp):
                adj = do_action(adj, (edge, edge_action))
                current_score = score
        if current_score > highest_score:
            highest_score = current_score
            best_adj = adj.copy()
    return best_adj

In [54]:
temps = np.arange(1000, 0, -1)
adj_annealing_bic = simulated_annealing(data, BIC, temps, 0)
np.save(f'submissions/annealing_bic.npy', adj_annealing_bic)
net = GaussNet(adj_annealing_bic).fit(data)
print(f'Likelihood: {net.loglikelihood(data)} with {net.num_params()} parameters.')

-0.0
0.01706010586990861
0.01471321983356591
0.04799529267174582
0.08611401738119655
0.006996381752893585
408.68550648660283
0.025650207589312298
0.1121154838489926
0.11462261571962165
38136.57495601694
38179.093198865165
38292.28327879202
38370.48729261235
38447.16317261772
38487.38943290525
38647.55491985753


/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykerne

38687.858523672265
34298.520410179095
34335.45926036059
34374.8643860063
34495.97114484493
34532.28294479935
34645.52286583783
34755.31993353784
34793.35072018296
0.009021225061730802
0.06460887644364557
0.0879600406455587
1.1630681994884984
1.1724904057307122
1.7552334558517537
13721.765036937124
181054.3536875358
181254.501490784


/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykerne

181645.02892698103
182036.92758701765
301474.45027874084
302451.44357506314
302779.6082323533
303108.3657726844
303437.8386981077
303768.2505566792
304762.14706521045
305099.37583527435
306086.0628609417
306758.7801910694
307872.27878707787
308216.3997516483
311506.6400391587


...

/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykerne

40918039.617804505
41967253.309411444
44235755.53264105
45464526.57602528
46763513.205247805
51132686.84933209
58437220.708501376
60601564.856787436
64344225.23301232
67141850.77202989
73536325.5798723
81277079.49359125
118788784.05947256
154425311.7985825


/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykerne

220601905.8965159
308841914.378677
386053335.4333581
514737796.4947918
1544212343.3092222
Likelihood: -1058054274.7694885 with 58 parameters.


/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
/tmp/ipykernel_99/197971214.py:56: RuntimeWarning: overflow encountered in exp
  if rng.random() < np.exp(- (score - highest_score) / temp):
